### Import dependencies for data import

Un-comment the code cell below to obtain the data. Blocks should be seperated to test whether they're individually working.

In [ ]:
'''
import requests
import tarfile
from os import path

# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

import pandas as pd 
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape

'''

Getting the features (these were obtained by reviewing the data by datatype, description from the US DoT(helped remove redudant features) on excel, removing unwanted columns, re-reading the data and converting them to the dictionary below)

In [ ]:
import pandas as pd

path = "airline_2m.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape

In [ ]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

In [ ]:
#commmented since featuredata CSV is available featuredata=df[featurecols]
'''
featuredata=featuredata.dropna()
featuredata.to_csv('featuredata.csv',index=False)
'''

In [ ]:
featuredata=pd.read_csv('featuredata.csv')
featuredata

In [ ]:
target_name = "Cancelled"
target = featuredata[target_name]
# note that all features are categorical in nature
data = featuredata.drop(columns=[target_name])

In [ ]:
target.value_counts()

Class imbalance might be an issue with models modelled on this data

In [ ]:
data.isnull().values.any()

Use a dummy classifier that predicts the majority class always as the base model

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(data, target)
dummy_preds=dummy_clf.predict(data)
dummy_clf.predict(data)
dummy_clf.score(data, target)

High accuracy,arising from the imbalanced classes in the target.

What's the performance of a logistic regression?

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, data, target, cv=10,return_estimator=True)

cv_results

array=cv_results["test_score"]
array.mean()